# FILTERING#
Here I am filtering for the plots that burned and have the specific measurement condition that they were measured on either side of a fire.  

#### What I have done here:
- Using WA_FIRE_PLOT_PAIRS, which was created in QGIS and contains all plots/fire pairs, I filter TREE by keeping rows with 'PLOT' that appear in WA_FIRE_PLOT_PAIRS.
    - **Note that this filtering is wrong right now because PLOT is not the unique indentifier.  Instead use a combination of PLOT, COUNTYCD, UNITCD, and STATECD** (STATECD unecessary in our case).  

- Then I add to WA_FIRE_PLOT_PAIRS the measurement years of those plots- this is done correctly as I use PLOT, COUNTYCD, UNITCD.  
- Now I can filter FIREPLOTS_MEAS (the plot/fire pairs with measurement years added) to only keep those that burned inbetween the measurement years.  
- **The final set FIREPLOTS_FIRE_SANDWICH only has these plot/fire pairs with burns inbetween measurements**

- **Now we need to see how many trees fall into these plots.** 

In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
sns.set_style("whitegrid")

Remember to unzip WA_TREE.csv.zip

In [ ]:
TREE = pd.read_csv('/Users/henrycladouhos/Desktop/WA_TREE.csv')
print(len(TREE))
TREE.columns

/var/folders/mm/t_p68_6506v3jg0m1f58zkv40000gn/T/ipykernel_66020/2301810624.py:1: DtypeWarning: Columns (77,159) have mixed types. Specify dtype option on import or set low_memory=False.
  TREE = pd.read_csv('/Users/henrycladouhos/Desktop/WA_TREE.csv')


504956


Index(['CN', 'PLT_CN', 'PREV_TRE_CN', 'INVYR', 'STATECD', 'UNITCD', 'COUNTYCD',
       'PLOT', 'SUBP', 'TREE',
       ...
       'VOLCSNET_BARK', 'DRYBIO_STEM', 'DRYBIO_STEM_BARK', 'DRYBIO_STUMP_BARK',
       'DRYBIO_BOLE_BARK', 'DRYBIO_BRANCH', 'DRYBIO_FOLIAGE',
       'DRYBIO_SAWLOG_BARK', 'PREV_ACTUALHT_FLD', 'PREV_HT_FLD'],
      dtype='object', length=197)

$N = 504,956$ 
- Now we can remove trees that are in plots that have not burned in the last 24 years.  
- To do this I will use **PLOT_FIRES_GIS**, which was created by intersecting the last 24 years of fire data from the NIFC with our plot latitude and longitude.  *We are introducing our first uncertainty here- the plot latitude and longitude have been 'fuzzed'.*

In [4]:
FIREPLOTS = pd.read_csv('../Data/DISSOLVED_FIRE_PLOT_PAIRS.csv')
print(len(FIREPLOTS))
print(FIREPLOTS.columns)


6168
Index(['CN', 'SRV_CN', 'CTY_CN', 'PREV_PLT_CN', 'INVYR', 'STATECD', 'UNITCD',
       'COUNTYCD', 'PLOT', 'PLOT_STATUS_CD', 'PLOT_NONSAMPLE_REASN_CD',
       'MEASYEAR', 'MEASMON', 'MEASDAY', 'REMPER', 'KINDCD', 'DESIGNCD',
       'RDDISTCD', 'WATERCD', 'LAT', 'LON', 'ELEV', 'GROW_TYP_CD',
       'MORT_TYP_CD', 'P2PANEL', 'P3PANEL', 'ECOSUBCD', 'CONGCD', 'MANUAL',
       'KINDCD_NC', 'QA_STATUS', 'CREATED_DATE', 'MODIFIED_DATE',
       'MICROPLOT_LOC', 'DECLINATION', 'EMAP_HEX', 'SAMP_METHOD_CD',
       'SUBP_EXAMINE_CD', 'MACRO_BREAKPOINT_DIA', 'INTENSITY', 'CYCLE',
       'SUBCYCLE', 'ECO_UNIT_PNW', 'TOPO_POSITION_PNW',
       'NF_SAMPLING_STATUS_CD', 'NF_PLOT_STATUS_CD',
       'NF_PLOT_NONSAMPLE_REASN_CD', 'P2VEG_SAMPLING_STATUS_CD',
       'P2VEG_SAMPLING_LEVEL_DETAIL_CD', 'INVASIVE_SAMPLING_STATUS_CD',
       'INVASIVE_SPECIMEN_RULE_CD', 'DESIGNCD_P2A', 'MANUAL_DB', 'SUBPANEL',
       'CONDCHNGCD_RMRS', 'FUTFORCD_RMRS', 'MANUAL_NCRS', 'MANUAL_NERS',
       'MANUAL_RMRS', 'PAC

**There are 7177 plots*fire pairs that have burned in the last 24 years.**
- Now we filter TREE by using these plots.  

In [5]:
TREE_IN_BURNED_PLOTS = TREE[TREE['PLOT'].isin(FIREPLOTS['PLOT'])]
print("The numer of tree measurements in fire plots is " + str(len(TREE_IN_BURNED_PLOTS)))


The numer of tree measurements in fire plots is 128183


In [102]:

# Step 1: Identify the first measurements (where PREV_TRE_CN is NaN)
first_measurement_trees = TREE_IN_BURNED_PLOTS[TREE_IN_BURNED_PLOTS['PREV_TRE_CN'].isna()]
first_measurement_trees = first_measurement_trees[['CN']].rename(columns={'CN': 'Root_CN'})

# Join first measurement with the subsequent measurements to get trees measured 2 times
second_measurement_trees = first_measurement_trees.merge(
    TREE_IN_BURNED_PLOTS, left_on='Root_CN', right_on='PREV_TRE_CN', suffixes=('', '_2'))

# Join again to get trees measured 3 times
third_measurement_trees = second_measurement_trees.merge(
    TREE_IN_BURNED_PLOTS, left_on='CN', right_on='PREV_TRE_CN', suffixes=('_2', '_3'))

# Step 3: Count trees measured exactly once, twice, three times, etc.
num_measured_once = len(first_measurement_trees) - len(second_measurement_trees)
num_measured_twice = len(second_measurement_trees) - len(third_measurement_trees)
num_measured_three_times = len(third_measurement_trees)

print("The number of trees measured once is:" +str(num_measured_once))
print("The number of trees measured twice is:"+ str(num_measured_twice))
print("The number of trees measured three times is:"+ str(num_measured_three_times))


The number of trees measured once is:18135
The number of trees measured twice is:54304
The number of trees measured three times is:0


$N = 128,183$
- This is still a lot of trees that appear in plots that have burned.  
- Now we need to filter for PLOTs that have been measured on either side of a burn
    - These measurements years are shown in the **WA_PLOT.csv** dataset in the *MEASYEAR* column

- **PLOT_FIRES_GIS** contains fire, plot pairs.  
    - Lets go through these pairs, referencing WA_PLOT to check if the measurement years are on either side of the fire year
        - This will filter **PLOT_FIRES_GIS** and will add the 2 years measured as columns to **PLOT_FIRES_GIS**.

In [103]:
PLOTS = pd.read_csv('../Data/WA_PLOT.csv')
PLOTS.columns

Index(['CN', 'SRV_CN', 'CTY_CN', 'PREV_PLT_CN', 'INVYR', 'STATECD', 'UNITCD',
       'COUNTYCD', 'PLOT', 'PLOT_STATUS_CD', 'PLOT_NONSAMPLE_REASN_CD',
       'MEASYEAR', 'MEASMON', 'MEASDAY', 'REMPER', 'KINDCD', 'DESIGNCD',
       'RDDISTCD', 'WATERCD', 'LAT', 'LON', 'ELEV', 'GROW_TYP_CD',
       'MORT_TYP_CD', 'P2PANEL', 'P3PANEL', 'ECOSUBCD', 'CONGCD', 'MANUAL',
       'KINDCD_NC', 'QA_STATUS', 'CREATED_DATE', 'MODIFIED_DATE',
       'MICROPLOT_LOC', 'DECLINATION', 'EMAP_HEX', 'SAMP_METHOD_CD',
       'SUBP_EXAMINE_CD', 'MACRO_BREAKPOINT_DIA', 'INTENSITY', 'CYCLE',
       'SUBCYCLE', 'ECO_UNIT_PNW', 'TOPO_POSITION_PNW',
       'NF_SAMPLING_STATUS_CD', 'NF_PLOT_STATUS_CD',
       'NF_PLOT_NONSAMPLE_REASN_CD', 'P2VEG_SAMPLING_STATUS_CD',
       'P2VEG_SAMPLING_LEVEL_DETAIL_CD', 'INVASIVE_SAMPLING_STATUS_CD',
       'INVASIVE_SPECIMEN_RULE_CD', 'DESIGNCD_P2A', 'MANUAL_DB', 'SUBPANEL',
       'CONDCHNGCD_RMRS', 'FUTFORCD_RMRS', 'MANUAL_NCRS', 'MANUAL_NERS',
       'MANUAL_RMRS', 'PAC_ISLA

In [104]:
FIREPLOTS.sample(3)

,CN,SRV_CN,CTY_CN,PREV_PLT_CN,INVYR,STATECD,UNITCD,COUNTYCD,PLOT,PLOT_STATUS_CD,...,PREV_PLOT_STATUS_CD_RMRS,REUSECD1,REUSECD2,REUSECD3,GRND_LYR_SAMPLING_STATUS_CD,GRND_LYR_SAMPLING_METHOD_CD,INCIDENT,FIRE_YEAR_,area,perimeter
1529,22398208010497,22132268010497,60010497,NaN,2002,53,6,9,60976,1,...,NaN,0.0,0.0,0.0,NaN,NaN,UNNAMED,1952,2.557060e+07,91859.194215
5924,174763490020004,44513110020004,82010497,2.239839e+13,2013,53,8,7,52175,1,...,NaN,NaN,NaN,NaN,NaN,NaN,Tyee,1994,5.752267e+08,161824.419606
1934,40220806010497,41062045010497,87010497,NaN,2010,53,9,23,55473,2,...,NaN,NaN,NaN,NaN,NaN,NaN,School,2005,2.081459e+08,117698.084696


In [105]:
FIREPLOTS_MEAS = FIREPLOTS.copy()
FIREPLOTS_MEAS['MEASYEAR1'] = None
FIREPLOTS_MEAS['MEASYEAR2'] = None
# This will just add MEASYEAR1 and MEASYEAR2
#After we will filter for rows where the fire occurs in between
nums_of_measurements = np.zeros(len(FIREPLOTS))
for index,row in FIREPLOTS_MEAS.iterrows():
    plot = row['PLOT']
    unitcd = row['UNITCD']
    countycd = row['COUNTYCD']
    
    uniqueplot = PLOTS[(PLOTS['PLOT']==plot) & (PLOTS['UNITCD'] == unitcd) & (PLOTS['COUNTYCD']==countycd)]
    if len(uniqueplot) >= 2:
        FIREPLOTS_MEAS.at[index,'MEASYEAR1'] = min(uniqueplot.MEASYEAR)
        FIREPLOTS_MEAS.at[index,'MEASYEAR2'] = max(uniqueplot.MEASYEAR)

FIREPLOTS_MEAS = FIREPLOTS_MEAS.dropna(subset = ['MEASYEAR1','MEASYEAR2'])

In [107]:
FIREPLOTS_FIRE_SANDWICH = FIREPLOTS_MEAS[(FIREPLOTS_MEAS['FIRE_YEAR_']>FIREPLOTS_MEAS['MEASYEAR1']) & (FIREPLOTS_MEAS['FIRE_YEAR_']<FIREPLOTS_MEAS['MEASYEAR2'])]
print(len(FIREPLOTS_FIRE_SANDWICH))
print(FIREPLOTS_FIRE_SANDWICH['PLOT'].nunique())

1596
579


**IF** I have done this properly there are 1596 fire, plot pairs that are sandwiched between two measurements.  These pairs are on 579 plots.  \
Let me check a couple rows..

In [109]:
FIREPLOTS_FIRE_SANDWICH[['MEASYEAR1','FIRE_YEAR_','MEASYEAR2']].sample(10)

,MEASYEAR1,FIRE_YEAR_,MEASYEAR2
18,2011,2012,2021
3514,2010,2015,2021
2543,2007,2010,2017
5507,2007,2012,2017
893,2010,2012,2020
5506,2006,2012,2016
411,2009,2015,2019
4918,2007,2010,2017
5821,2011,2014,2021
380,2010,2015,2020


**BANG!** \
Look at how those fires are sandwiched

OK.

I am claiming that we use this **FIREPLOTS_FIRE_SANDWICH** dataset to filter our trees.  This will give us our final dataset.  

PLEASE double check my work and logic behind the filtering.  

#### Lets see how many trees fall into these plots that have this measurement structure.  
- First I make a new column in **FIREPLOTS_FIRE_SANDWICH** and **TREE** which is just a concatenation of *UNITCD,COUNTYCD, and PLOT*.  IN THIS ORDER!
- The combination of these three columns is called 'UNIQUE_PLOT_ID'.

In [110]:
FIREPLOTS_FIRE_SANDWICH['UNIQUE_PLOT_ID'] =FIREPLOTS_FIRE_SANDWICH['UNITCD'].astype(str)+' '+FIREPLOTS_FIRE_SANDWICH['COUNTYCD'].astype(str)+' '+FIREPLOTS_FIRE_SANDWICH['PLOT'].astype(str)
TREE['UNIQUE_PLOT_ID'] = TREE['UNITCD'].astype(str)+' '+TREE['COUNTYCD'].astype(str)+' '+TREE['PLOT'].astype(str)

FIREPLOTS_FIRE_SANDWICH['UNIQUE_PLOT_ID'].nunique()

/var/folders/mm/t_p68_6506v3jg0m1f58zkv40000gn/T/ipykernel_66020/2003518706.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FIREPLOTS_FIRE_SANDWICH['UNIQUE_PLOT_ID'] =FIREPLOTS_FIRE_SANDWICH['UNITCD'].astype(str)+' '+FIREPLOTS_FIRE_SANDWICH['COUNTYCD'].astype(str)+' '+FIREPLOTS_FIRE_SANDWICH['PLOT'].astype(str)


579

In [111]:
TREES_FOR_US = TREE[TREE['UNIQUE_PLOT_ID'].isin(FIREPLOTS_FIRE_SANDWICH['UNIQUE_PLOT_ID'])]
len(TREES_FOR_US)

25518

$N=25518$ measurements! Now let's see how many are only measured once and get rid of those

In [112]:
first_measurement_trees_ofseries = TREES_FOR_US[TREES_FOR_US['CN'].isin(TREES_FOR_US['PREV_TRE_CN']) & TREES_FOR_US['PREV_TRE_CN'].isna()]
print("The number of trees/measurements that are first measurements of a series is " + str(first_measurement_trees_ofseries.shape[0]))
subsequent_measurement_trees = TREES_FOR_US[TREES_FOR_US['PREV_TRE_CN'].notna()]
print("The number of measurements that are subsequent measurements of a series is " + str(subsequent_measurement_trees.shape[0]))
multiple_measurement_cns = pd.concat([first_measurement_trees_ofseries['CN'], subsequent_measurement_trees['CN']])
single_measurement_trees = TREES_FOR_US[~TREES_FOR_US['CN'].isin(multiple_measurement_cns)]
print("The number of trees that were measured once is " + str(single_measurement_trees.shape[0]))

## Now find number of trees in each category
# Step 1: Identify the first measurements (where PREV_TRE_CN is NaN)
first_measurement_trees = TREES_FOR_US[TREES_FOR_US['PREV_TRE_CN'].isna()]
#first_measurement_trees = first_measurement_trees[['CN']].rename(columns={'CN': 'Root_CN'})

# Join first measurement with the subsequent measurements to get trees measured 2 times
second_measurement_trees = first_measurement_trees.merge(
    TREES_FOR_US, left_on='CN', right_on='PREV_TRE_CN', suffixes=('_1', ''))

# Join again to get trees measured 3 times
third_measurement_trees = second_measurement_trees.merge(
    TREES_FOR_US, left_on='CN', right_on='PREV_TRE_CN', suffixes=('_2', '_3'))


# Step 3: Count trees measured exactly once, twice, three times, etc.
num_measured_once = len(first_measurement_trees) - len(second_measurement_trees)
num_measured_twice = len(second_measurement_trees) - len(third_measurement_trees)
num_measured_three_times = len(third_measurement_trees)

print("The number of trees measured once is:" +str(num_measured_once))
print("The number of trees measured twice is:"+ str(num_measured_twice))
print("The number of trees measured three times is:"+ str(num_measured_three_times))

The number of trees/measurements that are first measurements of a series is 12059
The number of measurements that are subsequent measurements of a series is 12255
The number of trees that were measured once is 1204
The number of trees measured once is:1204
The number of trees measured twice is:12059
The number of trees measured three times is:0


Note that there are 196 measurements that were not captured in the trees measured once or twice categories. Let's dig into that.

In [114]:
# Look at trees not captured in above code
trees_captured_CN =pd.concat([first_measurement_trees['CN'], second_measurement_trees['CN']])
trees_notcaptured = TREES_FOR_US[~TREES_FOR_US['CN'].isin(trees_captured_CN)]
multiple_measurement_trees_notcaptured = trees_notcaptured[trees_notcaptured['CN'].isin(trees_notcaptured['PREV_TRE_CN'])]
misc_trees_notcaptured = trees_notcaptured[trees_notcaptured]
print("The number of trees measured twice in this uncaptured tree dataset is:" +str(len(multiple_measurement_trees_notcaptured)))


ValueError: Boolean array expected for the condition, not int64

The uncaptured trees were measured twice. My code didn't catch them since the first measurement did not have a null value for PREV_TRE_CN

In [115]:
#Get rid of trees measured once 

TREES_FOR_US_multipleMeasurements= TREES_FOR_US[TREES_FOR_US['CN'].isin(multiple_measurement_cns)]
print(len(TREES_FOR_US_multipleMeasurements))

24314


##### Adding fire and measurement info to TREES_FOR_US
- MEASYEAR1, MEASYEAR2
- FIRE YEAR (Multiple)
- FIRE SIZE


Whats the max number of fires that occured on a plot?


In [126]:
plot_numfires = []
for index, row in FIREPLOTS_FIRE_SANDWICH.iterrows():
    plot_name = row['UNIQUE_PLOT_ID']
    numfires = len(FIREPLOTS_FIRE_SANDWICH[FIREPLOTS_FIRE_SANDWICH['UNIQUE_PLOT_ID']==plot_name])

    plot_numfires.append([plot_name,numfires])

plot_numfires = np.array(plot_numfires)
plot_numfires = pd.DataFrame(plot_numfires, columns = ['UNIQUE_PLOT_ID','NUMFIRES'])
plot_numfires = plot_numfires.drop_duplicates()

plot_numfires['NUMFIRES'] = plot_numfires['NUMFIRES'].astype(int)
plot_numfires['UNIQUE_PLOT_ID'] = plot_numfires['UNIQUE_PLOT_ID'].astype(str)

print(plot_numfires['NUMFIRES'].min())
print(plot_numfires['NUMFIRES'].max())

plot_numfires['NUMFIRES'].idxmax()
plot_numfires.loc[13]

2
8


UNIQUE_PLOT_ID    8 47 79990
NUMFIRES                   8
Name: 13, dtype: object

In [127]:
FIREPLOTS_FIRE_SANDWICH[FIREPLOTS_FIRE_SANDWICH['UNIQUE_PLOT_ID'] == '8 47 79990']

,CN,SRV_CN,CTY_CN,PREV_PLT_CN,INVYR,STATECD,UNITCD,COUNTYCD,PLOT,PLOT_STATUS_CD,...,REUSECD3,GRND_LYR_SAMPLING_STATUS_CD,GRND_LYR_SAMPLING_METHOD_CD,INCIDENT,FIRE_YEAR_,area,perimeter,MEASYEAR1,MEASYEAR2,UNIQUE_PLOT_ID
30,29883386010497,29878201010497,92010497,NaN,2009,53,8,47,79990,2,...,NaN,NaN,NaN,Antoine 2,2012,2.767778e+07,28197.567994,2009,2019,8 47 79990
31,558629649126144,532456097126144,92010497,2.988339e+13,2019,53,8,47,79990,2,...,NaN,NaN,NaN,Antoine 2,2012,2.767778e+07,28197.567994,2009,2019,8 47 79990
2301,29883386010497,29878201010497,92010497,NaN,2009,53,8,47,79990,2,...,NaN,NaN,NaN,Reach,2015,3.497908e+08,231497.057459,2009,2019,8 47 79990
2322,558629649126144,532456097126144,92010497,2.988339e+13,2019,53,8,47,79990,2,...,NaN,NaN,NaN,Reach,2015,3.497908e+08,231497.057459,2009,2019,8 47 79990
4495,29883386010497,29878201010497,92010497,NaN,2009,53,8,47,79990,2,...,NaN,NaN,NaN,Carlton,2014,1.035210e+09,436020.430783,2009,2019,8 47 79990
4559,558629649126144,532456097126144,92010497,2.988339e+13,2019,53,8,47,79990,2,...,NaN,NaN,NaN,Carlton,2014,1.035210e+09,436020.430783,2009,2019,8 47 79990
5731,29883386010497,29878201010497,92010497,NaN,2009,53,8,47,79990,2,...,NaN,NaN,NaN,Carlton Complex,2014,1.020137e+09,407768.451876,2009,2019,8 47 79990
5792,558629649126144,532456097126144,92010497,2.988339e+13,2019,53,8,47,79990,2,...,NaN,NaN,NaN,Carlton Complex,2014,1.020137e+09,407768.451876,2009,2019,8 47 79990


The minimum number of fires on one plot is 1 and the maximum is 6.  Note that the 2 and 12 are multiplied by two because FIREPLOTS_FIRE_SANDWICH contains two measurements for each plot.  

Looking at the plot with this maximum number of fires, I NOTICE A PROBLEM...
- Each fire actually has multiple unconnected components
- So the deep harbor fire has three unconnected components that overlapped with the plot with UNIQUE_PLOT_ID = 8 7 66742.
- Not sure what to do with this

**OK, what I did was take the union of all the fire polygons**
Notice that now there are only two rows in the below dataset for each fire, one for each measurement.  

In [118]:
FIREPLOTS_FIRE_SANDWICH[FIREPLOTS_FIRE_SANDWICH['UNIQUE_PLOT_ID'] == '8 7 66742']

,CN,SRV_CN,CTY_CN,PREV_PLT_CN,INVYR,STATECD,UNITCD,COUNTYCD,PLOT,PLOT_STATUS_CD,...,REUSECD3,GRND_LYR_SAMPLING_STATUS_CD,GRND_LYR_SAMPLING_METHOD_CD,INCIDENT,FIRE_YEAR_,area,perimeter,MEASYEAR1,MEASYEAR2,UNIQUE_PLOT_ID
757,24126063010900,24120389010900,82010497,NaN,2002,53,8,7,66742,1,...,0.0,NaN,NaN,Deep Harbor,2004,1.235537e+08,75743.179625,2003,2012,8 7 66742
771,30764138020004,48466120010497,82010497,2.412606e+13,2012,53,8,7,66742,1,...,NaN,NaN,NaN,Deep Harbor,2004,1.235537e+08,75743.179625,2003,2012,8 7 66742
2155,24126063010900,24120389010900,82010497,NaN,2002,53,8,7,66742,1,...,0.0,NaN,NaN,Domke Lake,2007,4.768918e+07,47513.496931,2003,2012,8 7 66742
2157,30764138020004,48466120010497,82010497,2.412606e+13,2012,53,8,7,66742,1,...,NaN,NaN,NaN,Domke Lake,2007,4.768918e+07,47513.496931,2003,2012,8 7 66742


Lets add measyear1 and 2, fire year, fire size, and incident name.  
These last three may repeat up to 4 times (max times a plot burned in between measurements was 4 times)

In [125]:
TREES_FOR_US.columns
TREES_FOR_US.sample(3)

,CN,PLT_CN,PREV_TRE_CN,INVYR,STATECD,UNITCD,COUNTYCD,PLOT,SUBP,TREE,...,DRYBIO_STEM,DRYBIO_STEM_BARK,DRYBIO_STUMP_BARK,DRYBIO_BOLE_BARK,DRYBIO_BRANCH,DRYBIO_FOLIAGE,DRYBIO_SAWLOG_BARK,PREV_ACTUALHT_FLD,PREV_HT_FLD,UNIQUE_PLOT_ID
206619,15748173010497,40403375010497,NaN,2010,53,8,37,56865,4,10,...,60.961006,13.658029,1.055809,10.394316,38.294016,0.000000,NaN,NaN,NaN,8 37 56865
181852,799910010497,29882899010497,NaN,2009,53,8,7,63181,2,515,...,249.677450,41.422650,1.839692,36.625104,61.457811,23.452283,NaN,NaN,NaN,8 7 63181
497667,1143358071290487,750082881290487,4.498548e+13,2021,53,8,47,58569,2,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,45.0,8 47 58569


I think this code is working.  \
I know its not pretty.  \
It is adding measurement years and relevant fire information (up to 4 times!).  

In [141]:
TREES_FOR_US['MEASYEAR'] = None
columns_to_add = ['INCIDENT1', 'FIREYEAR1', 'AREA1', 'PERIM1','INCIDENT2', 'FIREYEAR2', 'AREA2', 'PERIM2','INCIDENT3', 'FIREYEAR3', 'AREA3', 'PERIM3','INCIDENT4', 'FIREYEAR4', 'AREA4', 'PERIM4']

for col in columns_to_add:
    TREES_FOR_US[col] = None
#initialize these new columns
for index1, row1 in TREES_FOR_US.iterrows():
    #Assign MEASYEAR1 and MEASYEAR2 
    pltid = row1['UNIQUE_PLOT_ID']
    firesonplot = FIREPLOTS_FIRE_SANDWICH[FIREPLOTS_FIRE_SANDWICH['UNIQUE_PLOT_ID']==pltid]
    if pd.isna(row1['PREV_TRE_CN']):
        TREES_FOR_US.loc[index1, 'MEASYEAR'] = firesonplot['MEASYEAR1'].iloc[0]
    else:
        #if we are on the second measurement lets add fire info as well
        TREES_FOR_US.loc[index1, 'MEASYEAR'] = firesonplot['MEASYEAR2'].iloc[0]
        
        i=1
        for index2, row2 in firesonplot.iterrows():
            if i==1 or TREES_FOR_US.loc[index1,f'INCIDENT{i-1}']!=row2['INCIDENT']:
                TREES_FOR_US.loc[index1,f'INCIDENT{i}'] = row2['INCIDENT']
                TREES_FOR_US.loc[index1,f'FIREYEAR{i}'] = row2['FIRE_YEAR_']
                TREES_FOR_US.loc[index1, f'AREA{i}'] = row2['area']
                TREES_FOR_US.loc[index1,f'PERIM{i}'] = row2['perimeter']
                i = i+1



/var/folders/mm/t_p68_6506v3jg0m1f58zkv40000gn/T/ipykernel_66020/1107872292.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TREES_FOR_US['MEASYEAR'] = None
/var/folders/mm/t_p68_6506v3jg0m1f58zkv40000gn/T/ipykernel_66020/1107872292.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TREES_FOR_US[col] = None


NOTE THAT: area and perim are in meters.

We can look at the trees that were in both the Deep Harbor fire and the Domke Lake fire.  

In [152]:
TREES_FOR_US[(TREES_FOR_US['INCIDENT1']=='Deep Harbor') & (TREES_FOR_US['INCIDENT2']=='Domke Lake')]

,CN,PLT_CN,PREV_TRE_CN,INVYR,STATECD,UNITCD,COUNTYCD,PLOT,SUBP,TREE,...,AREA3,PERIM3,INCIDENT4,FIREYEAR4,AREA4,PERIM4,INCIDENT5,FIREYEAR5,AREA5,PERIM5
251742,177512750020004,30764138020004,2.412608e+13,2012,53,8,7,66742,1,100,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
251743,177512751020004,30764138020004,2.412609e+13,2012,53,8,7,66742,1,101,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
251744,177512754020004,30764138020004,2.412609e+13,2012,53,8,7,66742,2,102,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
251745,177512755020004,30764138020004,2.412609e+13,2012,53,8,7,66742,2,103,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
251746,177512756020004,30764138020004,2.412609e+13,2012,53,8,7,66742,2,104,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
251747,177512757020004,30764138020004,2.412610e+13,2012,53,8,7,66742,2,105,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
251748,177512758020004,30764138020004,2.412610e+13,2012,53,8,7,66742,3,106,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
251749,177512759020004,30764138020004,2.412610e+13,2012,53,8,7,66742,3,107,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
251750,177512760020004,30764138020004,2.412610e+13,2012,53,8,7,66742,3,108,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
251751,177512761020004,30764138020004,2.412610e+13,2012,53,8,7,66742,3,109,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN


In [163]:
TREES_FOR_US[pd.isna(TREES_FOR_US['INCIDENT1'])]['PREV_TRE_CN'].info()

<class 'pandas.core.series.Series'>
Index: 13263 entries, 25391 to 504079
Series name: PREV_TRE_CN
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 207.2 KB


Quick check above: All of the trees with INCIDENT1 == None, are the first measurements of the trees. i.e. they havent burned yet and have no PREV_TRE_CN.  